In [ ]:
import pdfplumber
import pandas as pd
import re

# === CONFIG ===
pdf_path = r"C:\Users\wb611279\OneDrive - WBG\Desktop\AI Test\WBL_2026_Labor_Questionnaire2.pdf"
excel_path = r"C:\Users\wb611279\OneDrive - WBG\Desktop\AI Test\Fin_Extracted_WBL_Questions_Labor_FinalCleaned.xlsx"

topic = "Women, Business and the Law"
questions = []
current_section = None
section_instructions = {}
start_reading = False
policy_instruments_mode = False

# === POLICY INSTRUMENTS TEXT ===
policy_instruction_text = (
    "This final part focuses on policy instruments that support the implementation of laws. "
    "These may include national policies and action plans, institutions responsible for monitoring "
    "and enforcing laws, access to justice measures, government programs and services, and statistical data. "
    "Only instruments that are in place and operational should be considered. "
    "For the purpose of this Part, the “Source” field refers to the official source of information "
    "that can be used as a reference to substantiate the answer. Sources can include, for example, "
    "the title of an action plan and its issuing agency, the names of institutions, courts, police units, "
    "governmental agencies, or the title of sex-disaggregated datasets and their respective issuing agencies. "
    "Please provide links to sources where feasible."
)

# === REGEX PATTERNS ===
section_pattern = re.compile(r"^Section\s+\d+", re.IGNORECASE)
instruction_start_pattern = re.compile(r"^(Assumptions|Definitions)", re.IGNORECASE)
question_pattern = re.compile(r"^(\d+(\.\d+)*[A-Z]?)\.\s*(.*)", re.IGNORECASE)
start_pattern = re.compile(r"^1\.1\.", re.IGNORECASE)
skiplogic_pattern = re.compile(r"Skip Logic", re.IGNORECASE)
notes_trigger = re.compile(
    r"(?<!\?)\s*(ⓘ|Select\s|If\s+yes|If\s+no|include|includes|means|refers\s+to|should\s+be|such\s+as|for\s+example|where\s|when\s)",
    re.IGNORECASE,
)
filler_pattern = re.compile(r"(__.*?__|No change/Change|No change\s*/\s*Change)", re.IGNORECASE)
remove_after_question = re.compile(
    r"Answer\s*\(.*|Do you want to propose.*|Your updated answer.*|Legal Basis.*",
    re.IGNORECASE,
)
option_pattern = re.compile(r"•\s*(Employer|Government|Employer\s+and\s+Government)", re.IGNORECASE)

# === FUNCTIONS ===
def merge_multiline_text(lines, start_idx):
    merged = re.sub(filler_pattern, "", lines[start_idx]).strip()
    for j in range(start_idx + 1, len(lines)):
        nxt = re.sub(filler_pattern, "", lines[j]).strip()
        if not nxt:
            continue
        # Stop if a new question or section starts
        if question_pattern.match(nxt) or section_pattern.match(nxt):
            break
        merged += " " + nxt
    return merged.strip()


def clean_and_split_text(raw_text):
    """Clean question text and separate question from notes."""
    text = raw_text.strip()
    if not text:
        return "", ""
    text = re.sub(skiplogic_pattern, "", text)

    if "?" in text:
        question_part = text.split("?")[0].strip() + "?"
        remainder = "?".join(text.split("?")[1:]).strip()
    else:
        question_part = text.strip()
        remainder = ""

    question_part = re.sub(remove_after_question, "", question_part).strip()
    note = ""
    if notes_trigger.search(remainder):
        note = re.sub(r"ⓘ", "", remainder).strip()

    # Append Yes/No for binary questions
    if re.search(r"\b(does|is|are|can|has|should|do)\b", question_part, re.IGNORECASE):
        if not re.search(r"01[:]? ?Yes|00[:]? ?No", question_part):
            question_part = f"{question_part} 01: Yes | 00: No"

    return question_part, note


def extract_answer_options(block):
    """Extract bullet-style multiple-choice options (Employer, Government, etc.)."""
    options = option_pattern.findall(block)
    if options:
        opts = " | ".join([f"{str(i+1).zfill(2)}: {opt.strip()}" for i, opt in enumerate(options)])
        return opts
    return ""


# === PARSE PDF ===
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue
        lines = text.split("\n")

        for i, line in enumerate(lines):
            clean_line = line.strip()

            # detect start of Policy Instruments
            if "Policy Instruments – Instructions" in clean_line:
                policy_instruments_mode = True
                continue

            if section_pattern.match(clean_line):
                current_section = clean_line
                section_instructions[current_section] = ""
                continue

            if instruction_start_pattern.match(clean_line) and current_section:
                block = merge_multiline_text(lines, i)
                section_instructions[current_section] += " " + block
                continue

            if not start_reading:
                if start_pattern.match(clean_line):
                    start_reading = True
                else:
                    continue

            q_match = question_pattern.match(clean_line)
            if q_match:
                q_num = q_match.group(1)

                merged = merge_multiline_text(lines, i)
                # Skip garbage text (non-question patterns)
                if not re.search(r"\?\s*", merged):
                    continue
                if merged.startswith("3.1.1-3.1.3"):
                    continue

                # Extract options and clean question text
                options = extract_answer_options(merged)
                question_text, note = clean_and_split_text(merged)

                if options:
                    question_text = f"{question_text} {options}"

                parent = ".".join(q_num.split(".")[:-1]) if "." in q_num else None
                part_label = "Policy Instruments" if policy_instruments_mode else "Legal Framework"

                questions.append({
                    "Topic": topic,
                    "Section": current_section,
                    "Part": part_label,
                    "Instructions": section_instructions.get(current_section, "").strip(),
                    "Reference": f"WBL_{q_num.replace('.', '_')}",
                    "Question": question_text,
                    "ParentQuestion": f"WBL_{parent.replace('.', '_')}" if parent else "",
                    "QuestionNotes": note,
                })


# === BUILD DATAFRAME ===
df = pd.DataFrame(questions)

# === LABEL SECTIONS CORRECTLY ===
def update_section_title(row):
    sec = row["Section"]
    part = row["Part"]
    if "Work" in sec:
        return f"Section 1. Work - {part}"
    elif "Pay" in sec:
        return f"Section 2. Pay - {part}"
    elif "Parenthood" in sec:
        return f"Section 3. Parenthood - {part}"
    elif "Childcare" in sec:
        return f"Section 4. Childcare - {part}"
    elif "Pension" in sec:
        return f"Section 5. Pension - {part}"
    elif "Procurement" in sec:
        return "Section 6. Procurement - Legal Framework"
    else:
        return sec

df["Section"] = df.apply(update_section_title, axis=1)

# === ADD POLICY INSTRUMENTS TEXT ===
df.loc[df["Part"] == "Policy Instruments", "Instructions"] = policy_instruction_text

# === FINAL CLEAN FILTER ===
df = df[["Topic", "Section", "Instructions", "Reference", "Question", "ParentQuestion", "QuestionNotes"]]
df = df[df["Question"].str.len() > 10]  # remove empty/garbage
df.to_excel(excel_path, index=False)

print(f"✅ Cleaned and saved {len(df)} valid questions → {excel_path}")


In [ ]:
import pandas as pd

# === CONFIG ===
INPUT_FILE  = r"C:\Users\wb611279\OneDrive - WBG\Desktop\AI Test\Fin_Extracted_WBL_Questions_Labor_FinalCleaned.xlsx"
OUTPUT_FILE = r"C:\Users\wb611279\OneDrive - WBG\Desktop\AI Test\Clean_Fin_Extracted_WBL_Questions_Labor_FinalCleaned.xlsx"

# === 1. Load Excel ===
df = pd.read_excel(INPUT_FILE)

# === 2. Define special section rules ===
special_sections = {
    "Section 3. Parenthood - Legal Framework": 4,
    "Section 4. Childcare - Policy Instruments": 4
}

default_keep = 2

# === 3. Apply filtering ===
filtered = (
    df.groupby("Section", group_keys=False)
      .apply(lambda g: g.head(special_sections.get(g.name, default_keep)))
      .reset_index(drop=True)
)

# === 4. Save the filtered dataset ===
filtered.to_excel(OUTPUT_FILE, index=False)

print(f"✅ Filtered file saved to: {OUTPUT_FILE}")
print(f"Original rows: {len(df)}, Filtered rows: {len(filtered)}")
